<a href="https://colab.research.google.com/github/Yoshiki0418/Deep_Learning/blob/main/DL_Lecture2/deeper_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deeper Model

In [1]:
# googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Deep_Learning/DL_Lecture2

/content/drive/MyDrive/Deep_Learning/DL_Lecture2


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.nn import init
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch import nn, optim
import torchvision
from torchvision import transforms

%load_ext autoreload
%autoreload 2
import utils

### Residual Block

In [6]:
# 一番初期のresidual blockの実装
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()

        # スキップ結合時に次元が揃うようにする
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(X)
        out = F.relu(out)

In [7]:
# 改良版 (Pre-Activation)
class PreActBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.shortcut = nn.Sequential()

        # スキップ結合時に次元が揃うようにする
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            )

    def forward(self, X):
        out = self.conv1(F.relu(self.bn1(X)))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(X)
        return out

In [8]:
# インスタンス生成
resblock = ResidualBlock(3, 64, stride=2)
preact = PreActBlock(3, 64, stride=2)

In [9]:
preact

PreActBlock(
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (shortcut): Sequential(
    (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
  )
)

In [10]:
# 順伝搬テスト
X = torch.randn(1, 3, 32, 32)
output = preact(X)

In [11]:
output.shape

torch.Size([1, 64, 16, 16])

### Bottleneck構造

In [33]:
# point wiseは1x1のconv: パラメータ削減, 次元の変更に使う
# Bottleneck構造: 1x1conv -> 3x3conv -> 1x1convとすることで，channel数を減らして計算コストを削減する

class Bottleneck(nn.Module):
    def __init__(self, in_channel, out_channel, stride=1):
        super().__init__()
        self.expansion_factor = 4
        self.pw1 = nn.Conv2d(in_channel, int(out_channel/self.expansion_factor), kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(int(out_channel/self.expansion_factor))
        self.conv1 = nn.Conv2d(int(out_channel/self.expansion_factor), int(out_channel/self.expansion_factor), kernel_size=3, stride=stride, padding="same", bias=False)
        self.bn2 = nn.BatchNorm2d(int(out_channel/self.expansion_factor))
        self.pw2 = nn.Conv2d(int(out_channel/self.expansion_factor), out_channel, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channel)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, X):
        out = F.relu(self.bn1(self.pw1(X)))
        out = F.relu(self.bn2(self.conv1(out)))
        out = self.bn3(self.pw2(out))
        out += self.shortcut(X)
        out = F.relu(out)
        return out

In [34]:
# 順伝搬テスト
bottleneck = Bottleneck(256, 256, stride=1)
X = torch.randn(1, 256, 32, 32)
output = bottleneck(X)
output.shape

torch.Size([1, 256, 32, 32])

In [36]:
# 通常の畳み込みで同様の出力サイズにする場合は以下のような畳み込みを使用する（計算コストが高い
output2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)(X)

In [37]:
output2.shape

torch.Size([1, 256, 32, 32])

### Inception

In [40]:
class InceptionModule(nn.Module):
    def __init__(self, in_ch, out_ch1, out_ch3, out_ch5, out_ch_pool):
        super().__init__()

        # 1x1
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_ch, out_ch1, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_ch1),
            nn.ReLU()
        )

        # 3x3
        self.branch2 = nn.Sequential(
            # point-wise
            nn.Conv2d(in_ch, out_ch3, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_ch3),
            nn.ReLU(),
            # 3x3 conv
            nn.Conv2d(out_ch3, out_ch3, kernel_size=3, stride=1, padding="same", bias=False),
            nn.BatchNorm2d(out_ch3),
            nn.ReLU()
        )

        # 5x5
        self.branch3 = nn.Sequential(
            # point-wise
            nn.Conv2d(in_ch, out_ch5, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_ch5),
            nn.ReLU(),
            # 5x5 conv
            nn.Conv2d(out_ch5, out_ch5, kernel_size=5, padding="same", stride=1, bias=False),
            nn.BatchNorm2d(out_ch5),
            nn.ReLU(),
        )

        # pooling
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_ch, out_ch_pool, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_ch_pool),
            nn.ReLU()
        )

    def forward(self, X):
        branch1 = self.branch1(X)
        branch2 = self.branch2(X)
        branch3 = self.branch3(X)
        branch4 = self.branch4(X)

        return torch.cat([branch1, branch2, branch3, branch4], dim=1)

In [41]:
# InceptionModuleをテスト
X = torch.randn(16, 192, 28, 28)
module = InceptionModule(192, 64, 128, 32, 32)
out = module(X)
print(out.shape)

torch.Size([16, 256, 28, 28])


In [42]:
module

InceptionModule(
  (branch1): Sequential(
    (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (branch2): Sequential(
    (0): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (branch3): Sequential(
    (0): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=same, bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
